# Data Wrangling

In [63]:
# importing Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import sqlite3 as sql

In [64]:
# read dataFrame
df = pd.read_csv("df.csv", index_col="Unnamed: 0")

### Columns Manipulation

In [65]:
# "Semestre de ingresso no curso" to "Semestre de ingresso"
df["Semestre de ingresso"] = df["Semestre de ingresso no curso"]
df.drop(columns=["Semestre de ingresso no curso"], inplace=True)

# "Coeficiente de rendimento" to "CR"
df["CR"] = df["Coeficiente de rendimento"]
df.drop(columns=["Coeficiente de rendimento"], inplace=True)

# "Ano de ingresso" and "Semestre de ingresso" to "Ingresso"
df["Ingresso"] = df["Ano de ingresso"] + ((df["Semestre de ingresso"]-1)*0.5)
#df.drop(columns=["Ano de ingresso", "Semestre de ingresso"], inplace=True)

# "Ano" and "Semestre" to "Data"
df["Data"] = df["Ano"] + ((df["Semestre"]-1)*0.5)
df.drop(columns=["Ano", "Semestre"], inplace=True)

# "Mudou de curso - mesmo campus" and "Mudou de curso - outro campus" to "Mudou de Curso"
df["Mudou de curso"] = (df["Mudou de curso - mesmo campus"]=="Sim").astype(int) + 2*(df["Mudou de curso - outro campus"]=="Sim").astype(int)
df.drop(columns=["Mudou de curso - mesmo campus", "Mudou de curso - outro campus"], inplace=True)

# "Forma de ingresso" to "Sisu"
df["Sisu"] = df["Forma de ingresso"] == "SISU - Sistema de Seleção Unificada"

# "Data de nascimento" to "Ano de nascimento"
df["Ano de nascimento"] = df["Data de nascimento"].str[0:4].astype(int)
df.drop(columns=["Data de nascimento"], inplace=True)

# "Data" and "Ano de nascimento" to "Idade"
df["Idade"] = df["Data"].astype(int) - df["Ano de nascimento"]

# "Gênero" to "Mulher"
df["Mulher"] = df["Gênero"] == "Feminino"
df.drop(columns=["Gênero"], inplace=True)

# "Tipo de cota" to "Cotista"
df["Cotista"] = 1 - ((df["Tipo de cota"] == "Não cotista") + (df["Tipo de cota"] == "Sem cotas"))
df.drop(columns=["Tipo de cota"], inplace=True)

# "Data", "Ingresso", "Período" to "Atraso"
df["Atraso"] = 1 + 2*(df["Data"] - df["Ingresso"]) - df["Período"]

# "País de nascimento" to "Brasileiro"
df["Brasileiro"] = df["País de nascimento"] == "Brasil"
df.drop(columns=["País de nascimento"], inplace=True)

# "Cor ou raça" to "Branco"
df["Branco"] = df["Cor ou raça"] == "Branca"
df.drop(columns=["Cor ou raça"], inplace=True)

# "UF" to "Paraná"
df["Paraná"] = df["UF SISU"] == "PR"

# "Cidade" to "Curitiba"
df["Curitiba"] = df["Município SISU"] == "Curitiba"

# "Disciplinas aprovadas", "Disciplinas matriculadas" to "Disciplinas aprov/matr"
df["Disciplinas aprov/matr"] = df["Disciplinas aprovadas"]/df["Disciplinas matriculadas"]
df.replace([np.inf, -np.inf, np.nan], -1, inplace=True)
df["Disciplinas aprov/matr"] = (df["Disciplinas aprov/matr"]*20).astype(int)/20
df["Disciplinas aprov/matr"] = (df["Disciplinas aprov/matr"] > 1) * -1 + (df["Disciplinas aprov/matr"] <= 1) * df["Disciplinas aprov/matr"]
df.replace(-1, np.nan, inplace=True)

# "Disciplinas reprovadas por fequência", "Disciplinas reprovadas por nota" to "Disciplinas reprovadas"
df["Disciplinas reprovadas"] = df["Disciplinas reprovadas por frequência"] + df["Disciplinas reprovadas por nota"]

### Complex Columns

In [66]:
# "Disciplinas matriculadas" to "Disciplinas matriculadas acumuladas" using cumsum()
df["Disciplinas matriculadas acumuladas"] = df.sort_values("Data").groupby("Código")["Disciplinas matriculadas"].cumsum()

# "Disciplinas aprovadas" to "Disciplinas aprovadas acumuladas" using cumsum()
df["Disciplinas aprovadas acumuladas"] = df.sort_values("Data").groupby("Código")["Disciplinas aprovadas"].cumsum()

# "Disciplinas reprovadas por nota" to "Disciplinas reprovadas por nota acumuladas" using cumsum()
df["Disciplinas reprovadas por nota acumuladas"] = df.sort_values("Data").groupby("Código")["Disciplinas reprovadas por nota"].cumsum()

# "Disciplinas reprovadas por frequência" to "Disciplinas reprovadas por frequência acumuladas" using cumsum()
df["Disciplinas reprovadas por frequência acumuladas"] = df.sort_values("Data").groupby("Código")["Disciplinas reprovadas por frequência"].cumsum()

# "Disciplinas consignadas" to "Disciplinas consignadas acumuladas" using cumsum()
df["Disciplinas consignadas acumuladas"] = df.sort_values("Data").groupby("Código")["Disciplinas consignadas"].cumsum()

# "Disciplinas reprovadas" to "Disciplinas reprovadas acumuladas" using cumsum()
df["Disciplinas reprovadas acumuladas"] = df.sort_values("Data").groupby("Código")["Disciplinas reprovadas"].cumsum()

# "Disciplinas aprovadas acumuladas", "Disciplinas matriculadas acumuladas" to "Disciplinas aprov/matr acumuladas"
df["Disciplinas aprov/matr acumuladas"] = df["Disciplinas aprovadas acumuladas"]/df["Disciplinas matriculadas acumuladas"]
df.replace([np.inf, -np.inf, np.nan], -1, inplace=True)
df["Disciplinas aprov/matr acumuladas"] = (df["Disciplinas aprov/matr acumuladas"]*20).astype(int)/20
df["Disciplinas aprov/matr acumuladas"] = (df["Disciplinas aprov/matr acumuladas"] > 1) * -1 + (df["Disciplinas aprov/matr acumuladas"] <= 1) * df["Disciplinas aprov/matr acumuladas"]
df.replace(-1, np.nan, inplace=True)



### Droping data


In [67]:
df.dropna(axis=0, subset="CR", inplace=True)

### New Dataframes


In [68]:
# Select the first occurence of each student
first_df = df.sort_values(by=["Código", 'Data'], ascending=True)
first_df.drop_duplicates(subset=['Código'], keep='first', inplace=True)
# Selecting the last occurence of each student
last_df = df.sort_values(by=["Código", 'Data'], ascending=True)
last_df.drop_duplicates(subset=['Código'], keep='last', inplace=True)

### Others new Columns

In [69]:
# Column to check if the student ended the course
formados = last_df[last_df["Situação"].isin(["Formado", "Enade pendente"])]["Código"].to_list()
df["Formou"] = df['Código'].isin(formados)

# Column to check if the student left the course
evadidos = last_df[last_df["Situação"].isin(["Desistente", "Mudou de Curso", "Transferido", "Jubilado", "Falecido"])]["Código"].to_list()
df["Evadiu"] = df['Código'].isin(evadidos)

# Column to check if the student left the course in this semester
df["Evadiu nesse semestre"] = df["Situação"].isin(["Desistente", "Mudou de Curso", "Transferido", "Jubilado", "Falecido"])

# Column to check if the student ended the course in this semester
df["Formou nesse semestre"] = df["Situação"].isin(["Formado", "Enade pendente"])

In [70]:
df = df.sort_index(axis=1)

In [71]:
# Select the first occurence of each student
first_df = df.sort_values(by=["Código", 'Data'], ascending=True)
first_df.drop_duplicates(subset=['Código'], keep='first', inplace=True)
# Selecting the lasr occurence of each student
last_df = df.sort_values(by=["Código", 'Data'], ascending=True)
last_df.drop_duplicates(subset=['Código'], keep='last', inplace=True)
# Selecting the Students that left the course
left_df = df[df["Evadiu"] == 1]
# Selecting the Students that are late in th course
late_df = df[df["Atraso"] >= 1]
# Selecting the Students that are late in th course
allTrajetory_df = df[np.logical_or(df["Evadiu"] == 1, df["Formou"] == 1)]

In [72]:
c = sql.connect("db.db")
df.to_sql("df", con=c, if_exists="replace")
last_df.to_sql("last_df", con=c, if_exists="replace")
first_df.to_sql("first_df", con=c, if_exists="replace")
left_df.to_sql("left_df", con=c, if_exists="replace")
late_df.to_sql("late_df", con=c, if_exists="replace")
allTrajetory_df.to_sql("allTrajetory_df", con=c, if_exists="replace")

7524